# Expanding Madina
## Goals
* Implementing a betweenness flow sumulation function that just can be called from a zonal object with minimal parameters
* implementing a dunction that takews count data and outputs a list of statistical models and thwir interporetation
* set up a complete docstring template that's compliant and complete..
* use code to run sommerville, run stats and interpret
* publish the docstring on cityformlab.mit.edu or on github.com
* use code to runn Beirut, interpret results
* use mofel to run Melbourn, NYC,...

In [6]:
import sys
import pandas as pd
import geopandas as gpd
from datetime import datetime
from pathlib import Path
from shapely.ops import transform

sys.path.append("..")
from madina.zonal.zonal import Zonal
from madina.una.betweenness import parallel_betweenness
from madina.una.elastic import get_elastic_weight

In [7]:
class Logger():
    def __init__(self, output_folder):
        self.output_folder = output_folder
        self.start_time = datetime.now()
        self.log_df = pd.DataFrame(
            {
                "time": pd.Series(dtype='datetime64[ns]'),
                "distance": pd.Series(dtype="string"),
                "tune_penalty": pd.Series(dtype="string"),
                "elastic_weight": pd.Series(dtype="string"),
                "origin": pd.Series(dtype="string"),
                "destination": pd.Series(dtype="string"),
                "event": pd.Series(dtype="string")
            }
        )

    def simulation_start(self, zone, network_weight_settings, turn_penalty_settings, elastic_weight_settings):
        ##-------------------------------------------------------------------------------------------------------------< Betweenness Recode...
        self.betweenness_record = zone.layers['streets']['gdf'].copy(deep=True)
        self.separate_simulation_records = {}
        for network_weight in network_weight_settings:
            self.separate_simulation_records[network_weight] = {}
            for turn_penalty in turn_penalty_settings:
                self.separate_simulation_records[network_weight][turn_penalty] = {}
                for elastic_weight in elastic_weight_settings:
                    self.separate_simulation_records[network_weight][turn_penalty][elastic_weight] = self.betweenness_record.copy(deep=True)

    def log(self, input_dict):
        input_dict["time"] = datetime.now()
        if self.log_df.shape[0] == 0:
            print(f"total time\tseconds elapsed\tdiatance method\telastic_weight\t{'origin':^15s}\t{'destination':^15s}\tevent")
            input_dict["seconds_elapsed"] = 0
            input_dict["cumulative_seconds"] = 0
        else:
            time_elapsed = (input_dict["time"] - self.log_df.iloc[-1]["time"]).seconds
            input_dict["seconds_elapsed"] = time_elapsed
            input_dict["cumulative_seconds"] = self.log_df["seconds_elapsed"].sum() + time_elapsed

        for column_name in self.log_df.columns:
            if column_name not in input_dict:
                input_dict[column_name] = "---"

        self.log_df = self.log_df.append(input_dict, ignore_index=True)
        print(
            f"{input_dict['cumulative_seconds']:6.4f}\t\t"
            f"{input_dict['seconds_elapsed']}\t\t\t\t"
            f"{input_dict['distance']}\t\t"
            f"{input_dict['tune_penalty']}\t\t"
            f"{input_dict['elastic_weight']}\t\t"
            f"{input_dict['origin']:^15s}\t"
            f"{input_dict['destination']:^15s}\t"
            f"{input_dict['event']}"
        )

    def simulation_end(self, network_weight_settings, turn_penalty_settings, elastic_weight_settings):
        self.betweenness_record.to_csv(self.output_folder + "betweenness_record.csv")
        #self.betweenness_record.to_file(self.output_folder + "street_network_betweenness_record.geojson", driver="GeoJSON")
        for network_weight in network_weight_settings:
            for turn_penalty in turn_penalty_settings:
                for elastic_weight in elastic_weight_settings:
                    pairing_folder = self.output_folder + f"{network_weight}\\{'with_turns' if turn_penalty else 'no_turns'}\\{'elastic_weight' if elastic_weight else 'unadjusted_weight'}\\"
                    self.separate_simulation_records[network_weight][turn_penalty][elastic_weight].to_csv(pairing_folder+"betweenness_record.csv")
                    #self.separate_simulation_records[network_weight][turn_penalty][elastic_weight].to_file(pairing_folder+"street_network_betweenness_record.geojson", driver="GeoJSON")
        self.log({"event": "All DONE."})
        self.log_df.to_csv(self.output_folder + "time_log.csv")


In [1]:
def betweenness_flow_simulation(
        city_name="Shaqra",
        data_folder=None,
        output_folder=None,
        pairings_file="Pairings.csv",
        network_weight_settings = ["Perceived", "Geometric"],
        turn_penalty_settings = [False, True],
        elastic_weight_settings = [False, True],
        num_cores=8,
        turn_threshold_degree=45,
        turn_penalty_amount=30,
        impose_crs=None
    ):
    '''
    WHat used to be a major project, is now implemented into this standard workflow.....
    '''
    # Validate user input parameters, raise exceptions or make modifications..
    if data_folder is None:
        data_folder = "Cities\\"+city_name+"\\Data\\"
    if output_folder is None:
        start_time = datetime.now()
        output_folder = f"Cities\\{city_name}\\Simulations\\{start_time.year}-{start_time.month:02d}-{start_time.day:02d} {start_time.hour:02d}-{start_time.minute:02d}\\ "

    logger=Logger(output_folder)
    logger.log({"event": "beginning"})

    pairings = gpd.read_file(data_folder + pairings_file)

    shaqra = Zonal()
    if impose_crs is not None:
        shaqra = Zonal(projected_crs=impose_crs)

    shaqra.load_layer(
        layer_name='streets',
        file_path=data_folder +  pairings.at[0, "Network_File"]  # "Network.geojson"
    )
    if impose_crs is not None:
        shaqra.layers["streets"]["gdf"] = gpd.read_file(data_folder +  pairings.at[0, "Network_File"]).set_crs(impose_crs, allow_override=True)

    print(f"streets\t{shaqra.layers['streets']['gdf'].crs}")

    logger.simulation_start(shaqra, network_weight_settings, turn_penalty_settings, elastic_weight_settings)

    ##-------------------------------------------------------------------------------------------------------------< Data Cleaning/...
    geometry_gdf = shaqra.layers["streets"]["gdf"]
    polygon_idxs = geometry_gdf[geometry_gdf["geometry"].geom_type == "Polygon"].index
    geometry_gdf.loc[polygon_idxs,"geometry"] = geometry_gdf.loc[polygon_idxs, "geometry"].exterior
    shaqra.layers["streets"]["gdf"] = geometry_gdf

    if shaqra.layers["streets"]["gdf"].has_z.any():
        def _to_2d(x, y, z):
            return tuple(filter(None, [x, y]))
        shaqra.layers["streets"]["gdf"]["geometry"] = shaqra.layers["streets"]["gdf"][
            "geometry"].apply(lambda s: transform(_to_2d, s))

    if (shaqra.layers["streets"]["gdf"].geometry.geom_type == "MultiLineString").all():
        shaqra.layers["streets"]["gdf"]["geometry"] = shaqra.layers["streets"]["gdf"][
            "geometry"].apply(lambda s: s.geoms[0])

    # Setting up a street network
    create_street_nodes_edges(
        shaqra,
        source_layer="streets",
        flatten_polylines=False,
        node_snapping_tolerance=1,
        fuse_2_degree_edges=False,
        tolerance_angle=10,
        solve_intersections=False,
        loose_edge_trim_tolerance=0.001,
        weight_attribute=None if "Perceived" not in network_weight_settings else pairings.at[0, "Network_Cost"],
        discard_redundant_edges=True # <---------------------------------------TODO: Expose as a parameter
        )
    # This is to re-set the origins and destinations before any new iteration. TODO: implement as a Zonal function.
    clean_node_gdf = shaqra.layers["network_nodes"]["gdf"].copy(deep=True)

    # preparing percieved and geometric weights...
    perceived_network_weight = shaqra.layers["network_edges"]["gdf"]["weight"]
    perceived_network_weight = perceived_network_weight.apply(lambda x: max(1, x))      # To avoid any negative numbers...
    geometric_network_weight = shaqra.layers["network_edges"]["gdf"]["geometry"].length

    logger.log({"event": "Network topology created."})


    for idx, pairing in pairings.iterrows():
        # Loading layers,  if they're not already loaded.
        if pairing["Origin_Name"] not in shaqra.layers:
            shaqra.load_layer(
                layer_name=pairing["Origin_Name"],
                file_path=data_folder + pairing["Origin_File"]
            )
            if (impose_crs is not None) and (shaqra.layers[pairing["Origin_Name"]]['original_crs'] != impose_crs):
                shaqra.layers[pairing["Origin_Name"]]["gdf"] = gpd.read_file(data_folder + pairing["Origin_File"]).set_crs(impose_crs, allow_override=True)
                shaqra.layers[pairing["Origin_Name"]]['original_crs'] = impose_crs


            ##-------------------------------------------------------------------------------------------------------------< Data Cleaning/...
            if shaqra.layers[pairing["Origin_Name"]]["gdf"].has_z.any():
                shaqra.layers[pairing["Origin_Name"]]["gdf"]["geometry"] = shaqra.layers[pairing["Origin_Name"]]["gdf"]["geometry"].apply(
                    lambda s: transform(_to_2d, s))
        if pairing["Destination_Name"] not in shaqra.layers:
            shaqra.load_layer(
                layer_name=pairing["Destination_Name"],
                file_path=data_folder + pairing["Destination_File"]
            )
            if (impose_crs is not None) and (shaqra.layers[pairing["Destination_Name"]]['original_crs'] != impose_crs):
                shaqra.layers[pairing["Destination_Name"]]["gdf"] = gpd.read_file(data_folder + pairing["Destination_File"]).set_crs(impose_crs, allow_override=True)
                shaqra.layers[pairing["Destination_Name"]]['original_crs'] = impose_crs
            ##-------------------------------------------------------------------------------------------------------------< Data Cleaning/...
            if shaqra.layers[pairing["Destination_Name"]]["gdf"].has_z.any():
                shaqra.layers[pairing["Destination_Name"]]["gdf"]["geometry"] = shaqra.layers[pairing["Destination_Name"]]["gdf"][
                    "geometry"].apply(lambda s: transform(_to_2d, s))
        print(f"{pairing['Origin_Name']}\t{shaqra.layers[pairing['Origin_Name']]['gdf'].crs}")
        print(f"{pairing['Destination_Name']}\t{shaqra.layers[pairing['Destination_Name']]['gdf'].crs}")

        # making sure to clear any existing origins and destinations before adding new ones.
        shaqra.layers["network_nodes"]["gdf"] = clean_node_gdf.copy(deep=True)

        # iterating over network weight options..
        for network_weight in network_weight_settings:

            # setting the proper network_weight
            if network_weight == "Perceived":
                shaqra.layers["network_edges"]["gdf"]["weight"] = perceived_network_weight
            elif network_weight == "Geometric":
                shaqra.layers["network_edges"]["gdf"]["weight"] = geometric_network_weight

            # using an effecient insert algorithm TODO: should be built inti the main Madina code... currently imported from betweenness function..
            insert_nodes_v2(
                shaqra,
                label="origin",
                layer_name=pairing["Origin_Name"],
                weight_attribute=pairing["Origin_Weight"] if pairing["Origin_Weight"] != "Count" else None,
            )

            insert_nodes_v2(
                shaqra,
                label="destination",
                layer_name=pairing["Destination_Name"],
                weight_attribute=pairing["Destination_Weight"] if pairing["Destination_Weight"] != "Count" else None,
            )

            inelastic_weight = shaqra.layers["network_nodes"]["gdf"]['weight']


            logger.log({
                "origin": pairing["Origin_Name"],
                "destination": pairing["Destination_Name"],
                "event": "Origins and Destinations prepared."
                })

            shaqra.create_graph(light_graph=True, dense_graph=True)

            logger.log({
                "origin": pairing["Origin_Name"],
                "destination": pairing["Destination_Name"],
                "event": "Light and dense graphs prepared."
                })

            for turn_penalty in turn_penalty_settings:
                # TODO: Investigate the value of pasing internal calculations beween simulations..
                #retained_d_idxs = {}
                #retained_paths = {}
                #retained_distances = {}
                for elastic_weight in elastic_weight_settings:
                    # The order of these is important, as the weight is overriden by
                    # elastic weight as there is no clean way to update weight for now.
                    if elastic_weight:
                        get_elastic_weight(
                            shaqra,
                            search_radius=800,
                            detour_ratio=0.002,
                            beta=0.002,
                            decay=True,
                            turn_penalty=turn_penalty,
                            retained_d_idxs=None  #<------------------- This is very sensitive to the orfer of iteration. TODO: change to a more solid implementation
                            #retained_d_idxs=None
                            )

                        logger.log({
                            "origin": pairing["Origin_Name"],
                            "destination": pairing["Destination_Name"],
                            "event": "Elastic Weights generated.",
                            "distance": network_weight, "tune_penalty": turn_penalty,
                            "elastic_weight": elastic_weight})
                    else:
                        shaqra.layers["network_nodes"]["gdf"]['weight'] = inelastic_weight


                    node_gdf = shaqra.layers["network_nodes"]["gdf"]
                    origin_gdf = node_gdf[node_gdf["type"] == "origin"]

                    num_cores = min(origin_gdf.shape[0], num_cores) # if not elastic_weight else 1

                    betweenness_output = parallel_betweenness_2(
                        shaqra,
                        search_radius=float(pairing["Radius"]),
                        detour_ratio=float(pairing["Detour"]),
                        decay=False if elastic_weight else True,
                        decay_method="exponent",  # "power", "exponent"
                        beta=float(pairing["Beta"]),
                        path_detour_penalty="equal",  # "power", "exponent", "equal"
                        origin_weights=True,
                        closest_destination=False,
                        destination_weights=True,  # if pairing["Destination_Name"] != "Mosques" else False,
                        # perceived_distance=False,
                        num_cores=num_cores,
                        light_graph=True,
                        turn_penalty=turn_penalty,
                        #retained_d_idxs=retained_d_idxs if elastic_weight else None,
                        #retained_paths=retained_paths if elastic_weight else None,
                        #retained_distances=retained_distances if elastic_weight else None,
                        rertain_expensive_data=False if elastic_weight else True,
                        retained_d_idxs=None,
                        retained_paths=None,
                        retained_distances=None,
                        #rertain_expensive_data=False
                    )

                    if not elastic_weight: #< -------------------------------------------------------- sensitive to order of looping #TODO: implement for more general case.
                        retained_d_idxs = betweenness_output["retained_d_idxs"]
                        #retained_paths = betweenness_output["retained_paths"]
                        #retained_distances = betweenness_output["retained_distances"]

                    logger.log({
                        "origin": pairing["Origin_Name"],
                        "destination": pairing["Destination_Name"],
                        "event": "Betweenness estimated.",
                        "distance": network_weight,
                        "tune_penalty": turn_penalty,
                        "elastic_weight": elastic_weight})
                    
    logger.simulation_end(network_weight_settings, turn_penalty_settings, elastic_weight_settings)

ModuleNotFoundError: No module named 'madina'

In [ ]:
# Running simulations for 5 scenariios
for i in reversed(range(7)):
    print (f"----------------------------------Scenario {i}-----------------------")
    betweenness_flow_simulation(
        city_name="Beirut",
        data_folder=f'Cities\\Beirut\\scenario_data\\SCENARIO{i}\\',
        output_folder=f'Cities\\Beirut\\\scenario_output\\SCENARIO{i}\\',
        pairings_file="Pairings.csv",
        network_weight_settings=["Perceived"],          # ["Perceived", "Geometric"],
        turn_penalty_settings=[False],                   # [False, True]
        elastic_weight_settings=[True],          # [False, True]
        num_cores=48,
        turn_threshold_degree=45,
        turn_penalty_amount=30,
        impose_crs='epsg:32636'
    )

----------------------------------Scenario 6-----------------------
total time	seconds elapsed	diatance method	elastic_weight	    origin     	  destination  	event
0.0000		0				---		---		---		      ---      	      ---      	beginning
streets	epsg:32636
counter = 100, progress =  2.33
counter = 200, progress =  4.66
counter = 300, progress =  6.99
counter = 400, progress =  9.32
counter = 500, progress = 11.65
counter = 600, progress = 13.98
counter = 700, progress = 16.31
counter = 800, progress = 18.64
counter = 900, progress = 20.97
counter = 1000, progress = 23.30
counter = 1100, progress = 25.63
counter = 1200, progress = 27.96
counter = 1300, progress = 30.29
counter = 1400, progress = 32.62
counter = 1500, progress = 34.95
counter = 1600, progress = 37.28
counter = 1700, progress = 39.61
counter = 1800, progress = 41.94
counter = 1900, progress = 44.27
counter = 2000, progress = 46.60
counter = 2100, progress = 48.93
counter = 2200, progress = 51.26
counter = 2300, progress = 53.

In [2]:
    # Running simulations for 5 scenariios
for i in reversed(range(6)):
    print (f"----------------------------------Scenario {i}-----------------------")
    betweenness_flow_simulation(
        city_name="Beirut",
        data_folder=f'Cities\\Beirut\\scenario_data\\SCENARIO{i}\\',
        output_folder=f'Cities\\Beirut\\\scenario_output\\SCENARIO{i}\\',
        pairings_file="Pairings.csv",
        network_weight_settings=["Perceived"],          # ["Perceived", "Geometric"],
        turn_penalty_settings=[False],                   # [False, True]
        elastic_weight_settings=[True],          # [False, True]
        num_cores=48,
        turn_threshold_degree=45,
        turn_penalty_amount=30,
        impose_crs='epsg:32636'
    )

----------------------------------Scenario 5-----------------------
total time	seconds elapsed	diatance method	elastic_weight	    origin     	  destination  	event
0.0000		0				---		---		---		      ---      	      ---      	beginning
streets	epsg:32636
counter = 100, progress =  2.40
counter = 200, progress =  4.80
counter = 300, progress =  7.20
counter = 400, progress =  9.59
counter = 500, progress = 11.99
counter = 600, progress = 14.39
counter = 700, progress = 16.79
counter = 800, progress = 19.19
counter = 900, progress = 21.59
counter = 1000, progress = 23.99
counter = 1100, progress = 26.39
counter = 1200, progress = 28.78
counter = 1300, progress = 31.18
counter = 1400, progress = 33.58
counter = 1500, progress = 35.98
counter = 1600, progress = 38.38
counter = 1700, progress = 40.78
counter = 1800, progress = 43.18
counter = 1900, progress = 45.57
counter = 2000, progress = 47.97
counter = 2100, progress = 50.37
counter = 2200, progress = 52.77
counter = 2300, progress = 55.

In [3]:
betweenness_flow_simulation(
    city_name="Beirut",
    data_folder=f'Cities\\Beirut\\scenario_data\\original\\',
    output_folder=f'Cities\\Beirut\\\scenario_output\\original\\',
    pairings_file="Pairings.csv",
    network_weight_settings=["Perceived"],          # ["Perceived", "Geometric"],
    turn_penalty_settings=[False],                   # [False, True]
    elastic_weight_settings=[True],          # [False, True]
    num_cores=48,
    turn_threshold_degree=45,
    turn_penalty_amount=30,
    impose_crs='epsg:32636'
)

total time	seconds elapsed	diatance method	elastic_weight	    origin     	  destination  	event
0.0000		0				---		---		---		      ---      	      ---      	beginning
streets	epsg:32636


NotImplementedError: Multi-part geometries do not provide a coordinate sequence

In [7]:
def accisiility_simulation(
        city_name="Shaqra",
        data_folder=None,
        output_folder=None,
        pairings_file="Pairings.csv",
        network_weight_settings = ["Perceived", "Geometric"],
        turn_penalty_settings = [False, True],
        elastic_weight_settings = [False, True],
        num_cores=8,
        turn_threshold_degree=45,
        turn_penalty_amount=30,
        impose_crs=None
    ):
    '''
    WHat used to be a major project, is now implemented into this standard workflow.....
    '''
    # TODO: pass turn_penalty_threshold and egree all the way into the function "turn_penalty_value". Think of an appropriate way to either assume as a zonal.degree_threshold setting, or literally pass internally into each function?

    # Validate user input parameters, raise exceptions or make modifications..
    if data_folder is None:
        data_folder = "Cities\\"+city_name+"\\Data\\"
    if output_folder is None:
        start_time = datetime.now()
        output_folder = f"Cities\\{city_name}\\Simulations\\{start_time.year}-{start_time.month:02d}-{start_time.day:02d} {start_time.hour:02d}-{start_time.minute:02d}\\ "

    logger=Logger(output_folder)
    logger.log({"event": "beginning"})

    pairings = gpd.read_file(data_folder + pairings_file)

    shaqra = md.Zonal()
    if impose_crs is not None:
        shaqra = md.Zonal(projected_crs=impose_crs)

    shaqra.load_layer(
        layer_name='streets',
        file_path=data_folder +  pairings.at[0, "Network_File"]  # "Network.geojson"
    )
    if impose_crs is not None:
        shaqra.layers["streets"]["gdf"] = gpd.read_file(data_folder +  pairings.at[0, "Network_File"]).set_crs(impose_crs, allow_override=True)

    print(f"streets\t{shaqra.layers['streets']['gdf'].crs}")

    logger.simulation_start(shaqra, network_weight_settings, turn_penalty_settings, elastic_weight_settings)

    ##-------------------------------------------------------------------------------------------------------------< Data Cleaning/...
    geometry_gdf = shaqra.layers["streets"]["gdf"]
    polygon_idxs = geometry_gdf[geometry_gdf["geometry"].geom_type == "Polygon"].index
    geometry_gdf.loc[polygon_idxs,"geometry"] = geometry_gdf.loc[polygon_idxs, "geometry"].exterior
    shaqra.layers["streets"]["gdf"] = geometry_gdf

    if shaqra.layers["streets"]["gdf"].has_z.any():
        def _to_2d(x, y, z):
            return tuple(filter(None, [x, y]))
        shaqra.layers["streets"]["gdf"]["geometry"] = shaqra.layers["streets"]["gdf"][
            "geometry"].apply(lambda s: transform(_to_2d, s))


    if (shaqra.layers["streets"]["gdf"].geometry.geom_type == "MultiLineString").all():
        shaqra.layers["streets"]["gdf"]["geometry"] = shaqra.layers["streets"]["gdf"][
            "geometry"].apply(lambda s: s.geoms[0])

    # Setting up a street network
    create_street_nodes_edges(
        shaqra,
        source_layer="streets",
        flatten_polylines=False,
        node_snapping_tolerance=1,
        fuse_2_degree_edges=False,
        tolerance_angle=10,
        solve_intersections=False,
        loose_edge_trim_tolerance=0.001,
        weight_attribute=None if "Perceived" not in network_weight_settings else pairings.at[0, "Network_Cost"],
        discard_redundant_edges=True # <---------------------------------------TODO: Expose as a parameter
        )
    # This is to re-set the origins and destinations before any new iteration. TODO: implement as a Zonal function.
    clean_node_gdf = shaqra.layers["network_nodes"]["gdf"].copy(deep=True)

    # preparing percieved and geometric weights...
    perceived_network_weight = shaqra.layers["network_edges"]["gdf"]["weight"]
    perceived_network_weight = perceived_network_weight.apply(lambda x: max(1, x))      # To avoid any negative numbers...
    geometric_network_weight = shaqra.layers["network_edges"]["gdf"]["geometry"].length

    logger.log({"event": "Network topology created."})


    for idx, pairing in pairings.iterrows():
        # Loading layers,  if they're not already loaded.
        if pairing["Origin_Name"] not in shaqra.layers:
            shaqra.load_layer(
                layer_name=pairing["Origin_Name"],
                file_path=data_folder + pairing["Origin_File"]
            )
            if (impose_crs is not None) and (shaqra.layers[pairing["Origin_Name"]]['original_crs'] != impose_crs):
                shaqra.layers[pairing["Origin_Name"]]["gdf"] = gpd.read_file(data_folder + pairing["Origin_File"]).set_crs(impose_crs, allow_override=True)
                shaqra.layers[pairing["Origin_Name"]]['original_crs'] = impose_crs

            ##-------------------------------------------------------------------------------------------------------------< Data Cleaning/...
            if shaqra.layers[pairing["Origin_Name"]]["gdf"].has_z.any():
                shaqra.layers[pairing["Origin_Name"]]["gdf"]["geometry"] = shaqra.layers[pairing["Origin_Name"]]["gdf"]["geometry"].apply(
                    lambda s: transform(_to_2d, s))
        if pairing["Destination_Name"] not in shaqra.layers:
            shaqra.load_layer(
                layer_name=pairing["Destination_Name"],
                file_path=data_folder + pairing["Destination_File"]
            )
            if (impose_crs is not None) and (shaqra.layers[pairing["Destination_Name"]]['original_crs'] != impose_crs):
                shaqra.layers[pairing["Destination_Name"]]["gdf"] = gpd.read_file(data_folder + pairing["Destination_File"]).set_crs(impose_crs, allow_override=True)
                shaqra.layers[pairing["Destination_Name"]]['original_crs'] = impose_crs
            ##-------------------------------------------------------------------------------------------------------------< Data Cleaning/...
            if shaqra.layers[pairing["Destination_Name"]]["gdf"].has_z.any():
                shaqra.layers[pairing["Destination_Name"]]["gdf"]["geometry"] = shaqra.layers[pairing["Destination_Name"]]["gdf"][
                    "geometry"].apply(lambda s: transform(_to_2d, s))
        print(f"{pairing['Origin_Name']}\t{shaqra.layers[pairing['Origin_Name']]['gdf'].crs}")
        print(f"{pairing['Destination_Name']}\t{shaqra.layers[pairing['Destination_Name']]['gdf'].crs}")

        # making sure to clear any existing origins and destinations before adding new ones.
        shaqra.layers["network_nodes"]["gdf"] = clean_node_gdf.copy(deep=True)

        # iterating over network weight options..
        for network_weight in network_weight_settings:

            # setting the proper network_weight
            if network_weight == "Perceived":
                shaqra.layers["network_edges"]["gdf"]["weight"] = perceived_network_weight
            elif network_weight == "Geometric":
                shaqra.layers["network_edges"]["gdf"]["weight"] = geometric_network_weight

            # using an effecient insert algorithm TODO: should be built inti the main Madina code... currently imported from betweenness function..
            insert_nodes_v2(
                shaqra,
                label="origin",
                layer_name=pairing["Origin_Name"],
                weight_attribute=pairing["Origin_Weight"] if pairing["Origin_Weight"] != "Count" else None,
            )

            insert_nodes_v2(
                shaqra,
                label="destination",
                layer_name=pairing["Destination_Name"],
                weight_attribute=pairing["Destination_Weight"] if pairing["Destination_Weight"] != "Count" else None,
            )

            inelastic_weight = shaqra.layers["network_nodes"]["gdf"]['weight']


            logger.log({
                "origin": pairing["Origin_Name"],
                "destination": pairing["Destination_Name"],
                "event": "Origins and Destinations prepared."
                })

            shaqra.create_graph(light_graph=True, dense_graph=True)

            logger.log({
                "origin": pairing["Origin_Name"],
                "destination": pairing["Destination_Name"],
                "event": "Light and dense graphs prepared."
                })

            for turn_penalty in turn_penalty_settings:
                # TODO: Investigate the value of pasing internal calculations beween simulations..
                #retained_d_idxs = {}
                #retained_paths = {}
                #retained_distances = {}
                for elastic_weight in elastic_weight_settings:
                    # The order of these is important, as the weight is overriden by
                    # elastic weight as there is no clean way to update weight for now.
                    if elastic_weight:
                        get_elastic_weight(
                            shaqra,
                            search_radius=800,
                            detour_ratio=0.002,
                            beta=0.002,
                            decay=True,
                            turn_penalty=turn_penalty,
                            retained_d_idxs=None  #<------------------- This is very sensitive to the orfer of iteration. TODO: change to a more solid implementation
                            #retained_d_idxs=None
                            )

                        logger.log({
                            "origin": pairing["Origin_Name"],
                            "destination": pairing["Destination_Name"],
                            "event": "Elastic Weights generated.",
                            "distance": network_weight, "tune_penalty": turn_penalty,
                            "elastic_weight": elastic_weight})
                    else:
                        shaqra.layers["network_nodes"]["gdf"]['weight'] = inelastic_weight


                    node_gdf = shaqra.layers["network_nodes"]["gdf"]
                    origin_gdf = node_gdf[node_gdf["type"] == "origin"]
                    origin_gdf.rename(
                        columns={
                            "elastic_weight": f"elastic_weight_O({pairing['Origin_Name']})__D({pairing['Origin_Name']})_{network_weight}_{'with_turns' if turn_penalty else 'no_turns'}_{'elastic_weight' if elastic_weight else 'unadjusted_weight'})",
                            "gravity": f"gravity_O({pairing['Origin_Name']})__D({pairing['Origin_Name']})_{network_weight}_{'with_turns' if turn_penalty else 'no_turns'}_{'elastic_weight' if elastic_weight else 'unadjusted_weight'})",
                            "reach": f"reach_O({pairing['Origin_Name']})__D({pairing['Origin_Name']})_{network_weight}_{'with_turns' if turn_penalty else 'no_turns'}_{'elastic_weight' if elastic_weight else 'unadjusted_weight'})"
                        }
                    )
                    pairing_folder = output_folder + f"{network_weight}\\{'with_turns' if turn_penalty else 'no_turns'}\\{'elastic_weight' if elastic_weight else 'unadjusted_weight'}\\O({pairing['Origin_Name']})_D({pairing['Destination_Name']})\\"
                    origin_gdf.to_csv(pairing_folder + "reach_gravity_elastic_weight.csv")

In [8]:
for i in reversed(range(7)):
    print (f"----------------------------------Scenario {i}-----------------------")
    accisiility_simulation(
        city_name="Beirut",
        data_folder=f'Cities\\Beirut\\scenario_data\\SCENARIO{i}\\',
        output_folder=f'Cities\\Beirut\\\scenario_output\\SCENARIO{i}\\',
        pairings_file="Pairings.csv",
        network_weight_settings=["Perceived"],          # ["Perceived", "Geometric"],
        turn_penalty_settings=[False],                   # [False, True]
        elastic_weight_settings=[True],          # [False, True]
        num_cores=48,
        turn_threshold_degree=45,
        turn_penalty_amount=30,
        impose_crs='epsg:32636'
        )

----------------------------------Scenario 6-----------------------
total time	seconds elapsed	diatance method	elastic_weight	    origin     	  destination  	event
0.0000		0				---		---		---		      ---      	      ---      	beginning
streets	epsg:32636
counter = 100, progress =  2.33
counter = 200, progress =  4.66
counter = 300, progress =  6.99
counter = 400, progress =  9.32
counter = 500, progress = 11.65
counter = 600, progress = 13.98
counter = 700, progress = 16.31
counter = 800, progress = 18.64
counter = 900, progress = 20.97
counter = 1000, progress = 23.30
counter = 1100, progress = 25.63
counter = 1200, progress = 27.96
counter = 1300, progress = 30.29
counter = 1400, progress = 32.62
counter = 1500, progress = 34.95
counter = 1600, progress = 37.28
counter = 1700, progress = 39.61
counter = 1800, progress = 41.94
counter = 1900, progress = 44.27
counter = 2000, progress = 46.60
counter = 2100, progress = 48.93
counter = 2200, progress = 51.26
counter = 2300, progress = 53.

In [ ]:
# Running simulations for 5 scenariios
for i in range(7):
    print (f"----------------------------------Scenario {i}-----------------------")
    betweenness_flow_simulation(
        city_name="Beirut",
        data_folder='Cities\\Beirut\\scenario_data\\SCENARIO'+str(i)+"\\",
        output_folder='Cities\\Beirut\\\scenario_output\\SCENARIO'+str(i)+"\\",    
        #pairings_file="Pairings.csv",
        network_weight_settings=["Perceived"],          # ["Perceived", "Geometric"],
        turn_penalty_settings=[False],                   # [False, True]
        elastic_weight_settings=[True],          # [False, True]
        num_cores=48,
        turn_threshold_degree=45,
        turn_penalty_amount=30
    )

----------------------------------Scenario 0-----------------------
total time	seconds elapsed	diatance method	elastic_weight	    origin     	  destination  	event
0.0000		0				---		---		---		      ---      	      ---      	beginning
streets	EPSG:3857
counter = 100, progress =  2.40
counter = 200, progress =  4.80
counter = 300, progress =  7.20
counter = 400, progress =  9.60
counter = 500, progress = 12.00
counter = 600, progress = 14.40
counter = 700, progress = 16.80
counter = 800, progress = 19.20
counter = 900, progress = 21.60
counter = 1000, progress = 24.00
counter = 1100, progress = 26.40
counter = 1200, progress = 28.80
counter = 1300, progress = 31.20
counter = 1400, progress = 33.60
counter = 1500, progress = 36.00
counter = 1600, progress = 38.40
counter = 1700, progress = 40.80
counter = 1800, progress = 43.20
counter = 1900, progress = 45.60
counter = 2000, progress = 48.00
counter = 2100, progress = 50.40
counter = 2200, progress = 52.80
counter = 2300, progress = 55.2